In [110]:
import numpy as np 
import pandas as pd
import torch
from data_loader import *
from main import *
from tqdm import tqdm

# data = pd.read_csv("./data/KDDTrain+_20Percent.csv", header=None, names=['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login,is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','class'])
data = pd.read_csv("./data/KDDTrain+.csv", header=None, names=['duration', 'protocol_type','service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in','num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login','is_guest_login','count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'class'])
data.loc[data["class"] != "normal", 'class'] = 0
data.loc[data["class"] == "normal", 'class'] = 1
# data = pd.read_csv("kddcup.data_10_percent", header=None,names=['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'type'])
# data.loc[data["type"] != "normal.", 'type'] = 0
# data.loc[data["type"] == "normal.", 'type'] = 1

one_hot_protocol = pd.get_dummies(data["protocol_type"])
one_hot_service = pd.get_dummies(data["service"])
one_hot_flag = pd.get_dummies(data["flag"])

data = data.drop("protocol_type",axis=1)
data = data.drop("service",axis=1)
data = data.drop("flag",axis=1)

data = pd.concat([one_hot_protocol, one_hot_service, one_hot_flag, data], axis=1)
# print('Anomaly:{} ****** Normal:{}'.format(len(data[data['class']==0]), len(data[data['class']==1])))
data.head()

,icmp,tcp,udp,IRC,X11,Z39_50,aol,auth,bgp,courier,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,0,0,0,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1
1,0,0,1,0,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1
2,0,1,0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0
3,0,1,0,0,0,0,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1
4,0,1,0,0,0,0,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1


In [111]:
import random
idx_list = data[data['class']==1].index.tolist()
random.shuffle(idx_list)
data.drop(idx_list[:52630], inplace=True)

In [112]:
proportions = data["class"].value_counts()
print(proportions)
print("Anomaly Percentage", proportions[1] / proportions.sum())

0    58630
1    14713
Name: class, dtype: int64
Anomaly Percentage 0.2006053747460562


In [101]:
cols_to_norm = ["duration", "src_bytes", "dst_bytes", "wrong_fragment", "urgent", 
            "hot", "num_failed_logins", "num_compromised", "num_root", 
            "num_file_creations", "num_shells", "num_access_files", "count", "srv_count", 
            "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", 
            "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", 
            "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", 
            "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate" ]

# data.loc[:, cols_to_norm] = (data[cols_to_norm] - data[cols_to_norm].mean()) / data[cols_to_norm].std()
min_cols = data.loc[data["class"]==0 , cols_to_norm].min()
max_cols = data.loc[data["class"]==0 , cols_to_norm].max()

data.loc[:, cols_to_norm] = (data[cols_to_norm] - min_cols) / (max_cols - min_cols)
np.savez_compressed("kdd_cup",kdd=data.as_matrix())

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [108]:
class hyperparams():
    def __init__(self, config):
        self.__dict__.update(**config)
defaults = {
    'lr' : 1e-4,
    'num_epochs' : 200,
    'batch_size' : 1024,
    'gmm_k' : 4,
    'lambda_energy' : 0.1,
    'lambda_cov_diag' : 0.005,
    'pretrained_model' : None,
    'mode' : 'train',
    'use_tensorboard' : False,
    'data_path' : 'kdd_cup.npz',

    'log_path' : './dagmm/logs',
    'model_save_path' : './dagmm/models',
    'sample_path' : './dagmm/samples',
    'test_sample_path' : './dagmm/test_samples',
    'result_path' : './dagmm/results',

    'log_step' : 194//4,
    'sample_step' : 194,
    'model_save_step' : 194,
}

In [109]:
solver = main(hyperparams(defaults))
accuracy, precision, recall, f_score = solver.test()

DaGMM
DaGMM(
  (encoder): Sequential(
    (0): Linear(in_features=122, out_features=60, bias=True)
    (1): Tanh()
    (2): Linear(in_features=60, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=10, bias=True)
    (5): Tanh()
    (6): Linear(in_features=10, out_features=1, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=1, out_features=10, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=60, bias=True)
    (5): Tanh()
    (6): Linear(in_features=60, out_features=122, bias=True)
  )
  (estimation): Sequential(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): Tanh()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=10, out_features=4, bias=True)
    (4): Softmax()
  )
)
The number of parameters: 19267


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 28.45it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 26.16it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 27.63it/s]


======================TEST MODE======================
N: 29315
phi :
 tensor([0.2667, 0.2504, 0.1908, 0.2921], device='cuda:0',
       grad_fn=<DivBackward0>)
mu :
 tensor([[-0.1634,  1.1474,  0.0022],
        [-0.1632,  1.1474,  0.0019],
        [-0.1631,  1.1477,  0.0013],
        [-0.1632,  1.1477,  0.0013]], device='cuda:0', grad_fn=<DivBackward0>)
cov :
 tensor([[[ 2.2838e-04,  5.2960e-06, -2.3861e-04],
         [ 5.2960e-06,  6.4029e-04, -8.0257e-04],
         [-2.3861e-04, -8.0257e-04,  2.0541e-03]],

        [[ 2.3042e-04,  5.8547e-06, -2.4198e-04],
         [ 5.8547e-06,  6.3618e-04, -8.0199e-04],
         [-2.4198e-04, -8.0199e-04,  2.0581e-03]],

        [[ 2.3158e-04,  5.8545e-06, -2.4405e-04],
         [ 5.8545e-06,  6.3690e-04, -8.0216e-04],
         [-2.4405e-04, -8.0216e-04,  2.0583e-03]],

        [[ 2.3122e-04,  5.6529e-06, -2.4346e-04],
         [ 5.6529e-06,  6.3840e-04, -8.0230e-04],
         [-2.4346e-04, -8.0230e-04,  2.0567e-03]]], device='cuda:0',
       grad_f